## **LOCACIONES PLANTAS CLIENTES - ESCENARIOS**

In [8]:
import gurobipy as gp
from gurobipy import GRB
import pandas
import plotly.graph_objects as go

In [9]:
df_costo_cliente_planta = pandas.read_excel("./data.xlsx", sheet_name="COSTO CLIENTE-PLANTA", index_col=[0, 1])
df_demanda = pandas.read_excel("./data.xlsx", sheet_name="DEMANDAS CLIENTES", index_col=0)
df_plantas = pandas.read_excel("./data.xlsx", sheet_name="COSTO Y CAPACIDAD PLANTAS", index_col=0)

**Conjuntos**

**Conjuntos**
$$ P: \text{Plantas} = \{1, 2,...,30\} $$
$$ C: \text{Clientes} = \{1, 2,...,50\} $$

**Parámetros**
$$ \text{CostoApertura}_{p}: \text{Costo de abrir la planta } p\in P $$
$$ \text{Capacidad}_{p}: \text{Capacidad máxima de la planta } p\in P $$
$$ \text{Demanda}_{c}: \text{Demanda del cliente } c\in C $$
$$ \text{CostoAsignacion}_{p,c}: \text{Costo de asignar la planta } p\in P \text{ al cliente } c\in C $$

**Variables**
$$ X_{p} = \left\{ \begin{array}{cl}
1: \text{Si abro la planta } p \in P \\
0: \text{dlc}
\end{array} \right. $$
$$ Y_{pc} = \left\{ \begin{array}{cl}
1: \text{Si asigno el cliente } c \in C \text{ a la planta } p \in P \\
0: \text{dlc}
\end{array} \right. $$

**Función Objetivo**
$$ \text{minimizar } \text{FO} = \sum_{p\in P} \text{CAP}_{p} \times X_{p} + \sum_{p\in P} \sum_{c\in C} \text{CCP}_{pc} \times Y_{pc} $$


**Restricciones**
$$ \forall p\in P \, \forall c\in C: Y_{pc} \le X_{p} $$
$$ \forall p\in P: \sum_{c\in C} Y_{pc} \times D_{c} \le \text{CAPAC}_{p} $$
$$ \forall c\in C: \sum_{p\in P} Y_{pc} = 1 $$
$$ \forall p\in P: \sum_{c\in C} Y_{pc} \ge 3 X_{p}$$
$$ \forall p\in P: \sum_{c\in C} Y_{pc} \times D_{c} \ge 0.9 * \text{CAPAC}_{p} X_{p} $$

In [10]:
model = gp.Model()

PLANTAS = df_plantas.index.to_list()
CLIENTES = df_demanda.index.to_list()

x = model.addVars(PLANTAS, vtype=GRB.BINARY, name="x")
y = model.addVars([(planta, cliente) for planta in PLANTAS for cliente in CLIENTES], vtype=GRB.BINARY, name="y")

costo_abrir_plantas = gp.quicksum(x[planta] * df_plantas.loc[planta, "Costo de abrir"] for planta in PLANTAS)
costo_cliente_planta = gp.quicksum(y[(planta, cliente)] * df_costo_cliente_planta.loc[(planta, cliente), "Costo de asignar un cliente a un planta"] for planta in PLANTAS for cliente in CLIENTES)
model.setObjective(costo_abrir_plantas + costo_cliente_planta, GRB.MINIMIZE)

for planta in PLANTAS:
    for cliente in CLIENTES:
        model.addConstr(y[(planta, cliente)] <= x[planta], name=f"ct1_{planta}_{cliente}")
        
ctr_max_capac = model.addConstrs(
    (gp.quicksum(y[(planta, cliente)] * df_demanda.loc[cliente, "Demanda"] for cliente in CLIENTES) <= df_plantas.loc[planta, "Capacidad máxima"] for planta in PLANTAS),
    name=f"ct2_{planta}",
)

for cliente in CLIENTES:
    model.addConstr(gp.quicksum(y[(planta, cliente)] for planta in PLANTAS) == 1, name=f"ct3_{cliente}")

for planta in PLANTAS:
    model.addConstr(gp.quicksum(y[(planta, cliente)] for cliente in CLIENTES) >= 3 * x[planta], name=f"ct4_{planta}")

ctr_min_capac = model.addConstrs(
    (gp.quicksum(y[(planta, cliente)] * df_demanda.loc[cliente, "Demanda"] for cliente in CLIENTES) >= 0.9 * df_plantas.loc[planta, "Capacidad máxima"] * x[planta] for planta in PLANTAS),
    name=f"ct5_{planta}",
)

model.update()

In [11]:

# Limit how many solutions to collect
model.setParam(GRB.Param.PoolSolutions, 5)
# Limit the search space by setting a gap for the worst possible solution that will be accepted
model.setParam(GRB.Param.PoolGap, 0.2)

Set parameter PoolSolutions to value 5
Set parameter PoolGap to value 0.2


In [12]:
model.setParam("TimeLimit", 20)
model.optimize()

Set parameter TimeLimit to value 20
Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (win64 - Windows 11.0 (22631.2))

CPU model: Intel(R) Core(TM) i7-9750H CPU @ 2.60GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 1110 rows, 1020 columns and 6040 nonzeros
Model fingerprint: 0xbd23f1ae
Variable types: 0 continuous, 1020 integer (1020 binary)
Coefficient statistics:
  Matrix range     [1e+00, 9e+01]
  Objective range  [1e+00, 5e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+02]
Presolve removed 220 rows and 74 columns
Presolve time: 0.07s
Presolved: 890 rows, 946 columns, 6411 nonzeros
Variable types: 0 continuous, 946 integer (946 binary)

Root relaxation: objective 6.319329e+03, 1033 iterations, 0.06 seconds (0.04 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time


In [ ]:
model.SolCount

In [14]:
for idx in range(model.SolCount):
    model.setParam(GRB.Param.SolutionNumber, idx)
    print(f"##### Escenario {idx} #####")
    print(f"Funcion Objetivo: {model.PoolObjVal}")
    print(f"Plantas abiertas: {sum([x[planta].Xn for planta in PLANTAS])}")
    
    for planta in PLANTAS:
        if x[planta].Xn > 0.9:
            CLIENTES_ATENDIDOS = []
            for cliente in CLIENTES:
                if y[(planta, cliente)].Xn > 0.9:
                    CLIENTES_ATENDIDOS.append(cliente)
            print(f"=== Planta {planta}: {CLIENTES_ATENDIDOS}")
    
    print("\n\n")
            

##### Escenario 0 #####
Funcion Objetivo: 39726.0
Plantas abiertas: 15.0
=== Planta 3: [7, 20, 49]
=== Planta 4: [9, 10, 29]
=== Planta 6: [3, 47, 50]
=== Planta 7: [11, 35, 41]
=== Planta 9: [17, 19, 32, 36]
=== Planta 10: [4, 14, 43]
=== Planta 11: [8, 22, 37]
=== Planta 12: [6, 42, 48]
=== Planta 13: [13, 21, 28]
=== Planta 14: [15, 25, 33, 40]
=== Planta 15: [1, 5, 26]
=== Planta 16: [12, 30, 45, 46]
=== Planta 17: [31, 38, 39, 44]
=== Planta 18: [2, 23, 34]
=== Planta 19: [16, 18, 24, 27]



##### Escenario 1 #####
Funcion Objetivo: 39747.0
Plantas abiertas: 15.0
=== Planta 3: [7, 20, 49]
=== Planta 4: [9, 10, 29]
=== Planta 6: [3, 47, 50]
=== Planta 7: [11, 35, 41]
=== Planta 9: [17, 19, 32, 36]
=== Planta 10: [4, 14, 43]
=== Planta 11: [8, 22, 37]
=== Planta 12: [1, 6, 42]
=== Planta 13: [13, 21, 28]
=== Planta 14: [15, 25, 33, 40]
=== Planta 15: [5, 26, 48]
=== Planta 16: [12, 30, 45, 46]
=== Planta 17: [31, 38, 39, 44]
=== Planta 18: [2, 23, 34]
=== Planta 19: [16, 18, 24, 27]

In [15]:
print(f"##### Mejor Escenario #####")
print(f"Funcion Objetivo: {model.ObjVal}")
print(f"Plantas abiertas: {sum([x[planta].X for planta in PLANTAS])}")

for planta in PLANTAS:
    if x[planta].X > 0.9:
        CLIENTES_ATENDIDOS = []
        for cliente in CLIENTES:
            if y[(planta, cliente)].X > 0.9:
                CLIENTES_ATENDIDOS.append(cliente)
        print(f"=== Planta {planta}: {CLIENTES_ATENDIDOS}")

print("\n\n")

##### Mejor Escenario #####
Funcion Objetivo: 39726.0
Plantas abiertas: 15.0
=== Planta 3: [7, 20, 49]
=== Planta 4: [9, 10, 29]
=== Planta 6: [3, 47, 50]
=== Planta 7: [11, 35, 41]
=== Planta 9: [17, 19, 32, 36]
=== Planta 10: [4, 14, 43]
=== Planta 11: [8, 22, 37]
=== Planta 12: [6, 42, 48]
=== Planta 13: [13, 21, 28]
=== Planta 14: [15, 25, 33, 40]
=== Planta 15: [1, 5, 26]
=== Planta 16: [12, 30, 45, 46]
=== Planta 17: [31, 38, 39, 44]
=== Planta 18: [2, 23, 34]
=== Planta 19: [16, 18, 24, 27]



